# Setup

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

llm_model = "gpt-3.5-turbo"

# Working with Open AI Chat API

In [2]:
## Without Lang Chain we have to write more clunky code
def get_completion(prompt, model=llm_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]

In [3]:
get_completion("What is 1+1?")

'1+1 equals 2.'

In [ ]:
## Also using a template pattern has to be handle much more manually
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

style = """American English \
in a calm and respectful tone
"""

template_string_1 = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

get_completion(template_string_1)

"Ah, I'm really frustrated that my blender lid flew off and splattered my kitchen walls with smoothie! And to make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, friend."

# LangChain

## Models

In [30]:
from langchain_community.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=0.0, model=llm_model)

## Prompt Templates

In [7]:
from langchain.prompts import ChatPromptTemplate

In [ ]:
## LangChain Prompt Templates offer basically the same features as our manual approach above
## but scale better to longer prompts and dovetail nicely with the output parser features below

template_string_2 = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

prompt_template_1 = ChatPromptTemplate.from_template(template_string_2)

print("Prompt", prompt_template_1.messages[0].prompt)
print("Detected Input Variables:", prompt_template_1.messages[0].prompt.input_variables)

Prompt input_variables=['style', 'text'] input_types={} partial_variables={} template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n'
Detected Input Variables: ['style', 'text']


In [ ]:
## Formatting doesn't work too differently from an f-string

customer_style = """American English \
in a calm and respectful tone
"""

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

customer_messages = prompt_template_1.format_messages(
                    style=customer_style,
                    text=customer_email)

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [15]:
customer_response = chat(customer_messages)

customer_response.content

"Oh man, I'm really frustrated that my blender lid flew off and made a mess of my kitchen walls with smoothie! And on top of that, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, friend."

## Output Parsers

In [19]:
## Say we wanted the LLM to product structured output like:
## {
##   "gift": False,
##   "delivery_days": 5,
##   "price_value": "pretty affordable!"
## }

## We'd specify it in our prompt maybe like in the following situation
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

## This is a template that specifies the sturcture and has the format variables for user input
review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [21]:
## We do out normal completion and get a response
prompt_template_2 = ChatPromptTemplate.from_template(review_template)
messages = prompt_template_2.format_messages(text=customer_review)
response = chat(messages)

## No matter how it looks the response is a string and NOT json
## We need LangChain Output parsers to help us with that
print(response.content)
type(response.content)

{
    "gift": true,
    "delivery_days": 2,
    "price_value": "It's slightly more expensive than the other leaf blowers out there"
}


str

In [22]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [ ]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

# From the provied response schemas we can generate the format instructions for the template
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()

print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [26]:
## Now we can use these format instructions in a new template
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [28]:
response = chat(messages)
print(response.content)

```json
{
	"gift": true,
	"delivery_days": 2,
	"price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


In [29]:
## The output parser knows how to read this completion into the correct format
output_dict = output_parser.parse(response.content)

print(output_dict)
print(type(output_dict))

{'gift': True, 'delivery_days': 2, 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}
<class 'dict'>
